In [1]:
from LIMxCMBL.init import *
from LIMxCMBL.kernels import *

from jax import random
import sys

In [2]:
Lambda_idx = 18
n_bins = 100
ell_idx = 0

In [3]:
#SPHEREx
zmin = 5
zmax = 8

chimin = ccl.comoving_angular_distance(cosmo, 1/(1+zmin))
chimax = ccl.comoving_angular_distance(cosmo, 1/(1+zmax))

In [4]:
Lambda = Lambdas[Lambda_idx]

In [5]:
from scipy.interpolate import interp1d, LinearNDInterpolator
from scipy.integrate import quad, quad_vec, trapezoid

In [6]:
#CII
#_KI = np.array(KI)

#CO
#print('CO')
#_KI = np.array(KI_CO)

#Lya
print('Lya')
_KI = np.array(KI_Lya)

Lya


In [7]:
import jax
import jax.numpy as jnp
@jax.jit
def f_KILo(chi, external_chi, Lambda):
    return (Lambda / jnp.pi 
            * jnp.interp(x = chi, xp = chis, 
                         fp = _KI, left = 0, right = 0) 
            * jnp.sinc(Lambda * (external_chi - chi) / np.pi))

In [8]:
inner_dkparp_integral = np.load('/oak/stanford/orgs/kipac/users/delon/LIMxCMBL/inner_dkparp_integral.npy')
inner_dkparp_integral = jnp.array(inner_dkparp_integral.astype(np.float64))

from interpax import interp2d, interp1d

2025-03-14 15:54:22.778492: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.4 which is older than the ptxas CUDA version (12.6.85). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [9]:
@jax.jit
def f_aux(_chib, chi, chip):
    _delta = jnp.abs(1 - chi/_chib)
    _delta = jnp.where(_delta < 1e-6, 1e-6, 
                     jnp.where(_delta > 0.7, 0.7, _delta))

    cross_integrand_1 = (2 * jnp.interp(x = chi, xp = chis, fp = _KI, left = 0, right = 0) 
                       * interp2d(xq = _chib, yq=jnp.log(_delta), 
                           x = chibs, y = jnp.log(deltas), f=inner_dkparp_integral[ell_idx],
                           method='linear',) 
                       / (_chib**2))

    cross_integrand_1 *= f_KILo(2*_chib - chi, 
                                        external_chi = chip,
                                        Lambda=Lambda)
    return cross_integrand_1

@jax.jit
def f_integrand(_chib, chip, chi, ell_idx):
    #I Lo + Lo I
    
    

    cross_integrand_1 = jnp.where((chimin <= 2*_chib - chi) & (2*_chib - chi <= chimax),
                                  f_aux(_chib, chi, chip),
                                  0)
            
    cross_integrand_2 = jnp.where((chimin <= 2*_chib - chip) & (2*_chib - chip <= chimax),
                                  f_aux(_chib, chip, chi),
                                  0)
            
    cross_integrand = cross_integrand_1 + cross_integrand_2
    
    #LoLo
    plus = _chib*(1+deltas)
    mins = _chib*(1-deltas)
    _idxs = (chimin < plus) & (plus < chimax) & (chimin < mins) & (mins < chimax)
    
    LoLo_integrand  = jnp.where(_idxs,
                               f_KILo(plus, 
                                      external_chi = chi,
                                      Lambda=Lambda) 
                                * f_KILo(mins, 
                                         external_chi = chip,
                                         Lambda=Lambda),
                               0)
    LoLo_integrand += jnp.where(_idxs,
                               f_KILo(mins, 
                                      external_chi = chi,
                                      Lambda=Lambda) 
                                * f_KILo(plus, 
                                         external_chi = chip,
                                         Lambda=Lambda),0)
    LoLo_integrand *= (2 / _chib) #(d)
    LoLo_integrand *= deltas
    LoLo_integrand *= interp1d(xq = _chib,
                               x = chibs, f=inner_dkparp_integral[ell_idx],
                               method='linear',)
    
    LoLo_integrand = jnp.trapezoid(x = np.log(deltas), y = LoLo_integrand, axis=-1)
    return LoLo_integrand - cross_integrand

In [10]:
f_integrand(chimin+1, chimin+0.5, chimin+0.9, ell_idx)

Array(-6.6066954e-13, dtype=float32)

In [11]:
from scipy.integrate import nquad

In [12]:
chi_bin_edges = np.linspace(chimin*(1+1e-8), chimax*(1 - 1e-8), n_bins + 1)
dchi = np.mean(np.diff(chi_bin_edges))

In [13]:
bin1_idx = 0
bin2_idx = 0

In [14]:
l1, r1 = chi_bin_edges[bin1_idx], chi_bin_edges[bin1_idx + 1]
l2, r2 = chi_bin_edges[bin2_idx], chi_bin_edges[bin2_idx + 1]

In [15]:
options={'limit':100000, 'epsabs': 0.0, 'epsrel':1e-3}

In [16]:
from quadax import quadgk

In [ ]:
def chip_quad(chip, chi, b1, b2, ell_idx):
    bnds = jnp.hstack([jnp.linspace(b1, chimin,210), 
                       jnp.linspace(chimin, chimax, 9000),
                       jnp.linspace(chimax, b2,210)])
    return quadgk(f_integrand, bnds,
                  epsabs = 0.0, epsrel = 1e-3,
                  order = 61, max_ninter=10000,
                  args=(chip, chi, ell_idx))[0]

def chi_quad(chi, l2, r2, b1, b2, ell_idx):
    return quadgk(chip_quad, [l2, r2], 
                  epsabs = 0.0, epsrel = 1e-3,
                  order = 61, max_ninter=10000,
                  args=(chi, b1, b2, ell_idx))[0]

res, _ = quadgk(chi_quad, [l1, r1],
               epsabs=0.0, epsrel=1e-3,
               order=61, max_ninter=10000,
               args=(l2, r2, 10, chimax_sample, ell_idx))

/home/users/delon/.local/lib/python3.9/site-packages/jax/_src/core.py:678: FutureWarning: unhashable type: <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>. Attempting to hash a tracer will lead to an error in a future JAX release.
  warnings.warn(
/home/users/delon/.local/lib/python3.9/site-packages/jax/_src/core.py:678: FutureWarning: unhashable type: <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>. Attempting to hash a tracer will lead to an error in a future JAX release.
  warnings.warn(
/home/users/delon/.local/lib/python3.9/site-packages/jax/_src/core.py:678: FutureWarning: unhashable type: <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>. Attempting to hash a tracer will lead to an error in a future JAX release.
  warnings.warn(
/home/users/delon/.local/lib/python3.9/site-packages/jax/_src/core.py:678: FutureWarning: unhashable type: <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>. Attempting to hash a tracer will lea

In [ ]:
# res, err = nquad(f_integrand,
#                  [[10, chimax_sample],[l2, r2],[l1,r1]],
#                  args=(ell_idx,),
#                  opts=[options]*3)